In [1]:
import torch

!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-geometric


Looking in links: https://data.pyg.org/whl/torch-1.11.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 273.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.0-cp37-cp37m-linux_x86_64.whl size=318664 sha256=35c9bbd138b3ee5c9c1a76a1a9e756f945e3a611bcd51f35f19696611767b32d
  Stored in directory: /root/.cache/pip/wheels/2d/d1/15/8a2f0086896d156654a843fff4bdbeaf621cdd10310a0daad2
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 460.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.16-cp37-cp37m-linux_x86_64.whl size=625437 sha256=53d0a16b5a9a15494334b614b3881911bb88e84409d0294595ef4b383505bdfc
  Stored in directory: /root/.cache/pip/wheels/92/8d/17/1517bf0318b8343b665b7aadf906dff4bc8967d82cacc17992
Successfully built t

In [2]:
import numpy as np 
import pandas as pd 

df = pd.read_csv('../input/vkcup1/train_mod_v3.csv')
df

,Unnamed: 0,u,v,t,h,mask2drop
0,0,0,1,27,6,1
1,1,2,3,13,8,1
2,2,4,5,74,2,1
3,3,6,7,36,9,1
4,4,8,9,52,2,1
...,...,...,...,...,...,...
16745086,17414504,2369189,2300645,6,7,1
16745087,17414505,62448,88256,60,0,1
16745088,17414506,919913,1183054,71,0,1
16745089,17414507,1774,881,67,0,1


In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_test.to_csv('test_mod_v3.csv')
del df_test 
#del df

In [4]:
N_TOTAL = max(df['u'].max(), df['v'].max()) + 1
df

,Unnamed: 0,u,v,t,h,mask2drop
0,0,0,1,27,6,1
1,1,2,3,13,8,1
2,2,4,5,74,2,1
3,3,6,7,36,9,1
4,4,8,9,52,2,1
...,...,...,...,...,...,...
16745086,17414504,2369189,2300645,6,7,1
16745087,17414505,62448,88256,60,0,1
16745088,17414506,919913,1183054,71,0,1
16745089,17414507,1774,881,67,0,1


In [5]:
device = 'cpu'

In [6]:
import networkx as nx
import random
from networkx import ego_graph
from tqdm.auto import trange


class GraphSampler:
    def __init__(self, df, N_PARTS, number_same_graph = 10000, number_same_graph2 = 70000):
        self.df = df
        self.N_PARTS = N_PARTS
        self.number_same_graph = number_same_graph
        self.number_same_graph2 = number_same_graph2
        self.DF_TRAIN = []
        self.iteration = 0
        df1 = df
        for i in range(N_PARTS):
            df1, df_cur = train_test_split(df1, test_size = int(len(self.df) / N_PARTS), random_state=42)
            self.DF_TRAIN.append(df_cur)
        self.cur_graph = nx.Graph()

        self.edges_set = [set() for i in range(N_TOTAL)]
        for i in trange(self.df.shape[0]):
            u, v = self.df['u'].iloc[i], self.df['v'].iloc[i]
            self.edges_set[u].add(v)
            self.edges_set[v].add(u)

    def __getitem__(self, index):
        del self.cur_graph
        self.cur_graph = nx.Graph()
        self.cur_graph.add_nodes_from(list(range(N_TOTAL)))
        self.cur_graph.add_edges_from(self.DF_TRAIN[index][['u', 'v']].values)


    def get_random_subgraph(self, size_graph):
        mask = np.random.choice(self.indices, size=size_graph)
        return self.cur_graph.subgraph(self.indices[mask])

    def get_random_connected(self, thr=60):
        v0 = random.randint(0, N_TOTAL - 1)
        g0 = ego_graph(self.cur_graph, n=v0, radius=1)
        if len(g0.nodes) < thr:
            g0 = ego_graph(self.cur_graph, n=v0, radius=2)
        return g0
    
    def get_random_connected_fixed_start(self, v0, thr=60):
        i = 0
        while v0 not in self.cur_graph.nodes:
            self.__getitem__(i)
            i += 1
        g0 = ego_graph(self.cur_graph, n=v0, radius=1)
        return g0

    def generate_random_graph(self, p_connected, size_graph, thr=60):
        if self.iteration % self.number_same_graph == 0:
            self.__getitem__(random.randint(0, self.N_PARTS - 1))
        self.iteration += 1
        if random.random() < p_connected:
            return self.get_random_connected(thr)
        else:
            return self.get_random_subgraph(size_graph)

    @torch.no_grad()
    def inference_model(self, model, num_samples_big, degree_threshold, diameter, K, lim2evaluate = None):
        sum_apk, cnt_apk = 0, 1e-9
        n_vertexes = lim2evaluate if lim2evaluate is not None else N_TOTAL
        for i in trange(n_vertexes):
            if len(target_arr[i]) == 0:
                continue
            if self.iteration % self.number_same_graph2 == 0:
                self.__getitem__(random.randint(0, self.N_PARTS - 1))
            self.iteration += 1
            recommendations = []
            if len(self.edges_set[i]) >= degree_threshold:
                g0 = ego_graph(self.cur_graph, n=i, radius=diameter)
                vertexes = list(g0.nodes)
                id2pos = {val: pos for pos, val in enumerate(vertexes)}
                test_edges = []
            
                for v0 in vertexes:
                    if v0 not in self.edges_set[i] and i not in self.edges_set[v0]:
                        test_edges.append((i, v0))
                    
                output = model(g0)
                pred_logits = [(torch.sum(output[id2pos[v0]] * output[id2pos[v1]]).item(), ind)
                           for ind, (v0, v1) in enumerate(test_edges)]
                pred_logits = sorted(pred_logits)[::-1][:K]
                recommendations = [ind for _, ind in pred_logits]
            else:
                g0 = nx.Graph()
                g0.add_node(i)
                for _ in range(num_samples_big):
                     g0.add_node(random.randint(0, N_TOTAL - 1))
                vertexes = list(g0.nodes)
                id2pos = {val: pos for pos, val in enumerate(vertexes)}
                test_edges = []
            
                for v0 in vertexes:
                    if v0 not in self.edges_set[i] and i not in self.edges_set[v0]:
                        test_edges.append((i, v0))
                    
                output = model(g0)
                pred_logits = [(torch.sum(output[id2pos[v0]] * output[id2pos[v1]]).item(), ind)
                           for ind, (v0, v1) in enumerate(test_edges)]
                pred_logits = sorted(pred_logits)[::-1][:K]
                recommendations = [ind for _, ind in pred_logits]
            
            sum_apk += apk(recommendations, target_arr[i], K)
            cnt_apk += 1
        return sum_apk / cnt_apk
    

In [7]:
graph_manager = GraphSampler(df_train, 60)

  0%|          | 0/13396072 [00:00<?, ?it/s]

In [8]:
import torch 
import torch.nn as nn
import torch_geometric as tg
from torch_geometric.utils.convert import to_networkx, from_networkx

class DSSM(nn.Module):
    def __init__(self, num_vertexes, embedding_dim, hidden_dim, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(num_vertexes, embedding_dim)
        self.layers = nn.ModuleList([])
        self.layers.append(nn.Linear(embedding_dim, hidden_dim))
        self.relu = nn.ReLU()
        for i in range(num_layers):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim))
        self.last = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, graph):
        input_data = from_networkx(graph)
        input_vertexes = torch.tensor(list(graph.nodes)).to(device)
        hidden = self.embedding(input_vertexes)
        for layer in self.layers:
            hidden = layer(hidden)
            hidden = self.relu(hidden)
        return self.last(hidden)


In [9]:
from tqdm.auto import trange

model = torch.load('/kaggle/input/default-ngcf/start_model.pt', map_location='cpu').cpu()

vectors = [None for _ in range(N_TOTAL)]
for i in trange(model.embedding.weight.shape[0]):
    if vectors[i] is not None:
        continue
    cur_graph = graph_manager.get_random_connected_fixed_start(i)
    output = model(cur_graph)
    ind = -1
    nodes = list(cur_graph.nodes)
    for j in range(len(nodes)):
        vectors[nodes[j]] = output[j].cpu().detach().numpy()
vectors[:5]

  0%|          | 0/2522474 [00:00<?, ?it/s]

[array([ 2.10035592e-03, -5.30206785e-02, -1.45587057e-01, -3.78282741e-02,
        -5.43446019e-02, -1.24613531e-02,  1.23851463e-01, -1.37026072e-01,
         3.86417620e-02, -1.18383691e-02, -1.04333445e-01, -4.75835428e-02,
        -2.93575786e-02,  6.56702071e-02,  1.39514089e-01, -1.34952828e-01,
        -2.57464312e-03, -1.38507605e-01,  1.40072107e-02, -9.50008556e-02,
         1.41775887e-02, -1.49391638e-02, -5.01794815e-02, -7.44022056e-02,
        -5.79805449e-02, -6.30807206e-02,  8.49188641e-02,  3.14729661e-02,
        -1.92264766e-02,  1.11878134e-01, -1.96006093e-02,  1.48942292e-01,
        -5.59743084e-02,  2.83960700e-02,  5.14326654e-02, -5.39636612e-02,
         1.05810285e-01, -2.04773918e-02,  9.13997740e-03, -1.12286415e-02,
         1.11550100e-01, -5.51443025e-02, -6.51894212e-02,  1.75244734e-02,
        -1.67994555e-02, -2.68531386e-02,  7.50448480e-02,  5.05681746e-02,
         5.13083600e-02,  3.67240161e-02,  2.11970657e-02, -2.34860647e-02,
         1.0

In [10]:
with open("dssm.txt", "w") as f:
    f.write(f"{len(vectors)} {len(vectors[0])}\n")
    for i in range(len(vectors)):
        for j in range(len(vectors[0])):
            f.write(f"{round(vectors[i][j].item(), 7)}\n")